In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## ***Import modules***

In [ ]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

**See what's in label**

In [ ]:
# see what's in the label
df = pd.read_csv("/kaggle/input/car-parts-40-classes/car parts 50/car parts.csv")
df


In [ ]:
# Plotting the distribution of classes
plt.figure(figsize=(10, 6))
df['labels'].value_counts().plot(kind='bar')
plt.title('Distribution of Classes')
plt.xlabel('Class')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# **Define Model**

In [ ]:
# Load pre-trained MobileNetV2 model without the top (classification) layer
base_model = MobileNetV2(weights='imagenet', include_top=False)

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False
    
num_classes = 50

# Add new classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x) 

# Create the transfer learning model
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
model.summary()

# **Compile the model**

In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
              loss = "categorical_crossentropy",
              metrics = ['accuracy']
              )

# **Load and preprocess the Training dataset**

In [ ]:
# training dataset directory
train_dir = "/kaggle/input/car-parts-40-classes/car parts 50/train"

# target size 
target_size = (224, 224)

# batch size (for batch training)
batch_size = 32

# Define the data augmentation parameters
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range = 20,
                                   width_shift_range = 0.1,
                                   height_shift_range = 0.1,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   vertical_flip = False,
                                   fill_mode = 'nearest'
                                   )

# Load the dataset with data augmentation
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = target_size,
    batch_size = batch_size,
    class_mode='categorical'
)

**Display random images from train dataset with label**

In [ ]:
# Get a batch of images and labels
batch_images, batch_labels = next(train_generator)

# Display random images with labels
num_images = 10
random_indices = np.random.choice(batch_images.shape[0], num_images, replace=False)
random_images = batch_images[random_indices]
random_labels = batch_labels[random_indices]

# Display the random images with labels
plt.figure(figsize=(15, 5))
for i in range(num_images):
    plt.subplot(1, num_images, i+1)
    plt.imshow(random_images[i])
    plt.title(f"Label: {np.argmax(random_labels[i])}")  # Assuming one-hot encoded labels
    plt.axis('off')

plt.show()

# **Load and preprocess the Validation Dataset**

In [ ]:
# validation dataset directory
valid_dir = "/kaggle/input/car-parts-40-classes/car parts 50/valid"

# target size 
target_size = (224, 224)

# batch size (for batch training)
batch_size = 32

# rescale the validation dataset
valid_datagen = ImageDataGenerator(rescale=1./255)

# load the dataset
valid_generator = train_datagen.flow_from_directory(
    valid_dir,
    target_size = target_size,
    batch_size = batch_size,
    class_mode='categorical'
)

**Using EarlyStopping**

In [ ]:
# This History object contains the training history of the model, including the loss and any metrics specified during training.
history = model.fit(
    train_generator,
    epochs = 25,
    validation_data = valid_generator
)

In [ ]:
model.save('compressed_model.tflite')

# **Draw Learning Curve**

In [ ]:
# Access the loss values from the history object
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Plot the training and validation loss
plt.figure()
plt.plot(range(1, len(train_loss) + 1), train_loss, label='Training Loss')
plt.plot(range(1, len(val_loss) + 1), val_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

In [ ]:
# Access the accuracy values from the history object
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

# Plot the training and validation accuracy
plt.figure()
plt.plot(range(1, len(train_acc) + 1), train_acc, label='Training Accuracy')
plt.plot(range(1, len(val_acc) + 1), val_acc, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()